# Model Training

1. Retrieve all feature groups
2. Join features
3. Create a model per resort location (filter by location + sort)
4. Train XGBoost + validation
5. Check performance

In [2]:
import xgboost
import pandas as pd
import datetime
import hopsworks
import warnings
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import uniform, randint
from locations import *

warnings.filterwarnings("ignore", module="IPython")


In [3]:
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",             # DNS of your Hopsworks instance
    project="ID2223_Project"
)

fs = project.get_feature_store()

2026-01-12 09:44:47,399 INFO: Initializing external client
2026-01-12 09:44:47,400 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-12 09:44:48,203 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2173


## Create Feature View

In [4]:
warning_fg = fs.get_feature_group(
    name='avalanche_warning_with_lags',
    version=2
)

weather_fg = fs.get_feature_group(
    name="weather_terrain_sensor",
    version=2
)

In [6]:
selected_features = (
    warning_fg
        .select(["location", "date", "warning_level", "warning_level_lag_1", "warning_level_lag_2", "warning_level_lag_3"])
        .join(
            weather_fg.select([
                "temperature_2m_mean",
                "precipitation_sum",
                "rain_sum",
                "snowfall_sum",
                "wind_speed_10m_max",
                "wind_direction_10m_dominant",
                "snow_load_steep",
                "wind_snow_transport",
                "rain_on_snow_risk",
                "temp_elev",
                "precip_slope_weighted"]),
            on=["location", "date"]
        )
)
selected_features.features


[Feature('location', None, None, False, False, False, None, None, None),
 Feature('date', None, None, False, False, False, None, None, None),
 Feature('warning_level', None, None, False, False, False, None, None, None),
 Feature('warning_level_lag_1', None, None, False, False, False, None, None, None),
 Feature('warning_level_lag_2', None, None, False, False, False, None, None, None),
 Feature('warning_level_lag_3', None, None, False, False, False, None, None, None),
 Feature('temperature_2m_mean', None, None, False, False, False, None, None, None),
 Feature('precipitation_sum', None, None, False, False, False, None, None, None),
 Feature('rain_sum', None, None, False, False, False, None, None, None),
 Feature('snowfall_sum', None, None, False, False, False, None, None, None),
 Feature('wind_speed_10m_max', None, None, False, False, False, None, None, None),
 Feature('wind_direction_10m_dominant', None, None, False, False, False, None, None, None),
 Feature('snow_load_steep', None, Non

In [ ]:
feature_view = fs.create_feature_view(
    name="avalanche_warning_fv_new_corrected_more_features_and_lags",
    version=4,
    description="Feature view combining avalanche warnings, weather forecasts, and static terrain features for Norwegian ski resorts.",
    query=selected_features,
    labels=["warning_level"]
)

## Train-test splits

In [5]:
fv = fs.get_feature_view(
    name="avalanche_warning_fv_new_corrected_more_features_and_lags",
    version=3
)

In [6]:
start_date_test_data = "2025-06-30"
# Convert string to datetime object
test_start = datetime.datetime.strptime(start_date_test_data, "%Y-%m-%d")
X_train, X_test, y_train, y_test = fv.train_test_split(
    test_start=test_start
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.95s) 
2026-01-12 09:45:37,191 INFO: Computing insert statistics
2026-01-12 09:45:37,323 INFO: Computing insert statistics
2026-01-12 09:45:38,222 WARNING: VersionWarning: Incremented version to `6`.



In [7]:
print(len(X_train[X_train['location'] == 'Galdhøpiggen Summer Ski Centre'].sort_values(by='date')))
X_train[X_train['location'] == 'Galdhøpiggen Summer Ski Centre']

1640


,location,date,warning_level_lag_1,warning_level_lag_2,warning_level_lag_3,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,snow_load_steep,wind_snow_transport,rain_on_snow_risk,temp_elev,precip_slope_weighted
230,Galdhøpiggen Summer Ski Centre,2021-02-03 00:00:00+00:00,2,2,2,-19.653915,0.0,0.0,0.00,3.054701,212.763855,0.000000,0.588053,0.000000,-10.183571,0.000000
231,Galdhøpiggen Summer Ski Centre,2021-02-08 00:00:00+00:00,2,2,2,-21.889334,0.0,0.0,0.00,7.928177,223.419846,0.000000,1.526234,0.000000,-11.341841,0.000000
232,Galdhøpiggen Summer Ski Centre,2021-02-09 00:00:00+00:00,2,2,2,-21.543503,0.0,0.0,0.00,7.993297,215.749039,0.000000,1.538770,0.000000,-11.162651,0.000000
233,Galdhøpiggen Summer Ski Centre,2021-02-20 00:00:00+00:00,3,3,3,-2.324750,1.3,0.0,0.91,6.915374,50.984947,0.109549,1.331262,0.000000,-1.204557,24.578027
234,Galdhøpiggen Summer Ski Centre,2021-03-16 00:00:00+00:00,3,3,2,-6.147667,0.0,0.0,0.00,6.924738,205.346115,0.000000,1.333064,0.000000,-3.185381,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20942,Galdhøpiggen Summer Ski Centre,2025-05-27 00:00:00+00:00,2,3,2,5.391917,4.2,4.2,0.00,10.120099,108.325500,0.000000,1.948195,4.352415,2.793793,79.405919
20943,Galdhøpiggen Summer Ski Centre,2025-06-08 00:00:00+00:00,0,0,0,7.673166,4.1,3.8,0.21,8.647496,273.208221,0.025281,1.664708,3.937899,3.975810,77.515304
20944,Galdhøpiggen Summer Ski Centre,2025-06-11 00:00:00+00:00,0,0,0,6.614834,1.9,1.1,0.56,11.638917,294.712402,0.067415,2.240579,1.139918,3.427441,35.921731
20945,Galdhøpiggen Summer Ski Centre,2025-06-16 00:00:00+00:00,0,0,0,11.698166,0.1,0.1,0.00,14.861722,256.234772,0.000000,2.860993,0.103629,6.061342,1.890617


In [8]:
def prepare_data(X, y_enc, location):
    df = (
        X.assign(label=y_enc)
        .loc[X["location"] == location]
        .sort_values("date")
        .reset_index(drop=True)
    )

    X_loc = df.drop(columns=["location", "date", "label"])
    y_loc = df["label"].to_numpy()

    return X_loc, y_loc

In [ ]:
if os.path.exists("models") == False:
    os.mkdir("models")

if os.path.exists("confusion_matrices") == False:
    os.mkdir("confusion_matrices")

label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train.to_numpy().ravel())
y_test_enc  = label_encoder.transform(y_test.to_numpy().ravel())

param_distributions = {
    "n_estimators": randint(200, 1200),
    "max_depth": randint(3, 12),
    "learning_rate": uniform(0.01, 0.2),
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5),
    "gamma": uniform(0, 5)
}

metrics = {}

for loc in resorts.keys():
    
    print(f"Training and evaluating model for location: {loc}")

    X_train_loc, y_train_loc = prepare_data(X_train, y_train_enc, loc)
    X_test_loc,  y_test_loc  = prepare_data(X_test,  y_test_enc,  loc)

    tscv = TimeSeriesSplit(n_splits=5)

    model = xgboost.XGBClassifier(
        objective="multi:softprob",
        num_class=len(label_encoder.classes_),
        eval_metric="mlogloss",
        random_state=42
   )

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring="accuracy",
        cv=tscv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(X_train_loc, y_train_loc)
    best_model = random_search.best_estimator_
    
    best_model.save_model(f"models/xgb_model_{loc.replace(' ', '_')}.json")
    
    y_pred_enc = best_model.predict(X_test_loc)
    acc_score = accuracy_score(y_test_loc, y_pred_enc)
    metrics[loc] = acc_score

    cm = confusion_matrix(y_test_loc, y_pred_enc)
    print("Confusion Matrix:")
    print(cm)
    labels_unique = np.sort(np.unique(np.concatenate([y_test_loc, y_pred_enc])))
    cm_df = pd.DataFrame(cm, index=labels_unique, columns=labels_unique)
    cm_df.to_csv(f"confusion_matrices/confusion_matrix_class_{loc.replace(' ', '_')}.csv")

    print(f"Accuracy Score: {acc_score:.3f}")
       


Training and evaluating model for location: Narvik Ski Resort
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  0   9   5   0]
 [  1   1  17   1]
 [  0   0   5   4]]
Accuracy Score: 0.934
Training and evaluating model for location: Strandafjellet Skisenter
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  1  14   6   0]
 [  0   2  15   1]
 [  0   0   1   3]]
Accuracy Score: 0.944
Training and evaluating model for location: Voss Resort Fjellheisar
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  1  17   4   0]
 [  0   1  13   3]
 [  0   0   2   2]]
Accuracy Score: 0.944
Training and evaluating model for location: Myrkdalen Fjellandsby
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  1  16   5   0]
 [  0   1  11   5]
 [  0   1   2   1]]
Accuracy Score: 0.924
Training and evalua

In [31]:
if os.path.exists("models") == False:
    os.mkdir("models")

if os.path.exists("confusion_matrices") == False:
    os.mkdir("confusion_matrices")

y_train_ord = y_train.to_numpy().ravel().astype(float)
y_test_ord  = y_test.to_numpy().ravel().astype(float)


param_distributions = {
    "n_estimators": randint(200, 1200),
    "max_depth": randint(3, 12),
    "learning_rate": uniform(0.01, 0.2),
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5),
    "gamma": uniform(0, 5)
}

for loc in resorts.keys():

    print(f"\nTraining and evaluating model for location: {loc}")

    X_train_loc, y_train_loc = prepare_data(X_train, y_train_ord, loc)
    X_test_loc,  y_test_loc  = prepare_data(X_test,  y_test_ord,  loc)

    tscv = TimeSeriesSplit(n_splits=5)

    model = xgboost.XGBRegressor(
        objective="reg:squarederror",
        eval_metric="rmse",
        random_state=42
    )

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring="neg_mean_absolute_error",
        cv=tscv,
        n_jobs=-1,
        verbose=2,
        random_state=42
    )

    random_search.fit(X_train_loc, y_train_loc)
    best_model = random_search.best_estimator_

    # Save model
    model_path = f"models/xgb_ordinal_model_{loc.replace(' ', '_')}.json"
    best_model.save_model(model_path)

    # Predict (continuous)
    y_pred_cont = best_model.predict(X_test_loc)

    # Convert to ordinal classes
    y_pred_ord = np.round(y_pred_cont)
    y_pred_ord = np.clip(y_pred_ord, 0, 5)

    # Metrics
    mae = mean_absolute_error(y_test_loc, y_pred_ord)
    acc = accuracy_score(y_test_loc, y_pred_ord)
    cm = confusion_matrix(y_test_loc, y_pred_ord)
    # print(y_pred_ord)
    # print(y_test_ord)
    print("Confusion Matrix:")
    print(cm)
    labels = np.sort(
        np.unique(
            np.concatenate([y_test_loc, y_pred_ord])
        )
    )
    print(labels)
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)
    cm_df.to_csv(f"confusion_matrices/confusion_matrix_reg_{loc.replace(' ', '_')}.csv")

    print(f"MAE: {mae:.3f}")
    print(f"Rounded Accuracy: {acc:.3f}")


Training and evaluating model for location: Narvik Ski Resort
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  0   9   5   0]
 [  1   1  17   1]
 [  0   0   4   5]]
[0. 1. 2. 3.]
MAE: 0.066
Rounded Accuracy: 0.939

Training and evaluating model for location: Strandafjellet Skisenter
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  1  14   6   0]
 [  0   2  15   1]
 [  0   0   1   3]]
[0. 1. 2. 3.]
MAE: 0.056
Rounded Accuracy: 0.944

Training and evaluating model for location: Voss Resort Fjellheisar
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  1  17   4   0]
 [  0   1  13   3]
 [  0   0   2   2]]
[0. 1. 2. 3.]
MAE: 0.056
Rounded Accuracy: 0.944

Training and evaluating model for location: Myrkdalen Fjellandsby
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Confusion Matrix:
[[154   0   0   0]
 [  1  16   5 

### Evaluation

#### Regression Model

In [24]:
scores_reg = {}

for loc in resorts.keys():
    print(f"\nEvaluating model for location: {loc} on training set")

    # Load the saved model
    model_path = f"models/xgb_ordinal_model_{loc.replace(' ', '_')}.json"
    best_model = xgboost.XGBRegressor()
    best_model.load_model(model_path)

    # Prepare training data for the location
    X_train_loc, y_train_loc = prepare_data(X_train, y_train_ord, loc)

    # Predict (continuous)
    y_pred_cont = best_model.predict(X_train_loc)

    # Convert to ordinal classes
    y_pred_ord = np.round(y_pred_cont)
    y_pred_ord = np.clip(y_pred_ord, 0, 5)

    # Confusion Matrix
    cm = confusion_matrix(y_train_loc, y_pred_ord)
    labels = np.sort(np.unique(np.concatenate([y_train_loc, y_pred_ord])))
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)
    
    print("Confusion Matrix:")
    print(cm_df)
    print(accuracy_score(y_train_loc, y_pred_ord))

    scores_reg[loc] = (accuracy_score(y_train_loc, y_pred_ord), cm_df)


Evaluating model for location: Narvik Ski Resort on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  755    6    0    0    0
1.0    1  103   46    0    0
2.0    1   25  463   21    0
3.0    0    0   63  127    3
4.0    0    0    0    8   18
0.8939024390243903

Evaluating model for location: Strandafjellet Skisenter on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  756    5    0    0    0
1.0    1  116   49    0    0
2.0    3   17  382   55    0
3.0    0    0   54  188    0
4.0    0    0    0   13    1
0.8798780487804878

Evaluating model for location: Voss Resort Fjellheisar on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  751    9    1    0    0
1.0    1  156   37    0    0
2.0    2   21  374   46    0
3.0    0    0   53  174    1
4.0    0    0    0    6    8
0.8920731707317073

Evaluating model for location: Myrkdalen Fjellandsby on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  755    6    0    0    0
1.0    1  

#### Classification Model

In [25]:
scores_class = {}
for loc in resorts.keys():
    print(f"\nEvaluating model for location: {loc} on training set")

    # Load the saved model
    model_path = f"models/xgb_model_{loc.replace(' ', '_')}.json"
    best_model = xgboost.XGBClassifier()
    best_model.load_model(model_path)

    # Prepare training data for the location
    X_train_loc, y_train_loc = prepare_data(X_train, y_train_ord, loc)

    # Predict (continuous)
    y_pred_cont = best_model.predict(X_train_loc)

    # Convert to ordinal classes
    y_pred_ord = np.round(y_pred_cont)
    y_pred_ord = np.clip(y_pred_ord, 0, 5)

    # Confusion Matrix
    cm = confusion_matrix(y_train_loc, y_pred_ord)
    labels = np.sort(np.unique(np.concatenate([y_train_loc, y_pred_ord])))
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)
    
    print("Confusion Matrix:")
    print(cm_df)
    print(accuracy_score(y_train_loc, y_pred_ord))
    scores_class[loc] = (accuracy_score(y_train_loc, y_pred_ord), cm_df)


Evaluating model for location: Narvik Ski Resort on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  756    4    1    0    0
1.0    1   99   50    0    0
2.0    2   19  459   30    0
3.0    1    1   68  119    4
4.0    0    0    0   11   15
0.8829268292682927

Evaluating model for location: Strandafjellet Skisenter on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  756    5    0    0    0
1.0    1  117   48    0    0
2.0    3   10  399   45    0
3.0    0    0   45  197    0
4.0    0    0    0    9    5
0.8987804878048781

Evaluating model for location: Voss Resort Fjellheisar on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  756    3    2    0    0
1.0    1  158   35    0    0
2.0    3   17  389   34    0
3.0    0    0   42  185    1
4.0    0    0    0    5    9
0.9128048780487805

Evaluating model for location: Myrkdalen Fjellandsby on training set
Confusion Matrix:
     0.0  1.0  2.0  3.0  4.0
0.0  756    3    2    0    0
1.0    1  

#### Comparison

In [28]:
if os.path.exists("confusion_matrix_images") == False:
    os.mkdir("confusion_matrix_images")

labels = np.arange(0, 5)

def plot_confusion_matrix(ax, cm, title, labels):
    # Convert DataFrame → NumPy array if needed
    if hasattr(cm, "values"):
        cm = cm.values

    im = ax.imshow(cm)

    ax.set_title(title)
    ax.set_xlabel("Predicted label")
    ax.set_ylabel("True label")

    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

    thresh = cm.max() / 2.0 if cm.max() > 0 else 0

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j, i, int(cm[i, j]),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black"
            )

    return im


for resort in resorts.keys():

    acc_class, cm_class = scores_class[resort]
    acc_reg, cm_reg     = scores_reg[resort]

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    plot_confusion_matrix(
        axes[0],
        cm_class,
        f"Classifier\nAccuracy = {acc_class:.3f}",
        labels
    )

    plot_confusion_matrix(
        axes[1],
        cm_reg,
        f"Ordinal Regressor\nAccuracy = {acc_reg:.3f}",
        labels
    )

    fig.suptitle(f"Confusion Matrices — {resort}", fontsize=14)

    plt.tight_layout()
    plt.savefig(
        f"confusion_matrix_images/confusion_matrices_{resort.replace(' ', '_')}.png",
        dpi=300
    )
    plt.close(fig)


## Save to model registry

In [15]:
model_registry = project.get_model_registry()
#use hash of location name to create unique model name
for loc in resorts.keys():
    loc_ = loc.replace(' ', '_')
    hash_loc = abs(hash(loc_)) % (10 ** 8)
    model_path = f"models/xgb_model_{loc_}.json"
    model_ = model_registry.python.create_model(
        name=f"xgb_avalanche_model_{hash_loc}",
        description=f"XGBoost model for avalanche warning level prediction at {loc}",
        feature_view=fv,
        metrics ={"accuracy": metrics[loc]},
    )
    model_.save(model_path)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\klara\Universidade\Year 2\Scalable ML and DL\Project\ID2223-Project\models/xgb_model_Narvik…

Uploading c:\Users\klara\Universidade\Year 2\Scalable ML and DL\Project\ID2223-Project\model_schema.json: 0.00…

Model created, explore it at https://eu-west.cloud.hopsworks.ai:443/p/2173/models/xgb_avalanche_model_55417835/1


  0%|          | 0/6 [00:00<?, ?it/s]

...

Model created, explore it at https://eu-west.cloud.hopsworks.ai:443/p/2173/models/xgb_avalanche_model_92380510/1


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\klara\Universidade\Year 2\Scalable ML and DL\Project\ID2223-Project\models/xgb_model_Bjorli…

Uploading c:\Users\klara\Universidade\Year 2\Scalable ML and DL\Project\ID2223-Project\model_schema.json: 0.00…

Model created, explore it at https://eu-west.cloud.hopsworks.ai:443/p/2173/models/xgb_avalanche_model_13148616/1
